In [1]:
import numpy as np
import pandas as pd
import keras
from dataset_generator import TimeWindowDatasetGenerator

from sklearn.calibration import LabelEncoder
from sklearn.discriminant_analysis import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from sklearn.model_selection import train_test_split

from tensorflow.keras.layers import Dropout


2024-03-21 19:17:28.580388: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-21 19:17:28.583240: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-21 19:17:28.615900: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-21 19:17:29.455179: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
data_generator = TimeWindowDatasetGenerator()
labeled_data = data_generator.get_labelled_timewindow_dataframe(student_data_filepath='task/student_data.csv',
                                                                time_window=10,
                                                                label_feature='affect',
                                                                exclude_after_notification=False,
                                                                exclude_after_engagement=False)


labeled_data

,sessionId,timestamp,x,y,z,ppgValue,hr,hrIbi,hrStatus,ibiStatus,notification,engagement,affect,context,label_id,label
12763,1,493272,884.0,1564.0,3767.0,2103379.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,RELAXED
12764,1,493311,900.0,1518.0,3782.0,2104145.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,RELAXED
12765,1,493351,894.0,1553.0,3734.0,2104953.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,RELAXED
12766,1,493391,907.0,1488.0,3729.0,2105398.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,RELAXED
12767,1,493431,861.0,1559.0,3731.0,2105114.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,RELAXED
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8101814,58,6184903,453.0,552.0,4126.0,2126027.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,240,HAPPY
8101815,58,6184943,494.0,607.0,4059.0,2126361.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,240,HAPPY
8101816,58,6184983,695.0,841.0,4124.0,2127234.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,240,HAPPY
8101817,58,6185023,497.0,591.0,4329.0,2128049.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,240,HAPPY


In [3]:
df = labeled_data
def replace_nan_hr(row):
    if not pd.isna(row['hr']) and row['hrStatus'] != 1:  
        mean_hr = df[(df['hrStatus'] == 1)]['hr'].median()
        return mean_hr
    return row['hr']
def replace_nan_hrIbi(row):
    if not pd.isna(row['hrIbi']) and row['ibiStatus'] != 0:  
        mean_hr = df[(df['ibiStatus'] == 1)]['hrIbi'].median()
        return mean_hr
    return row['hrIbi']


In [4]:
labeled_data['hr'] = df.apply(replace_nan_hr, axis=1)
labeled_data['hrIbi'] = df.apply(replace_nan_hrIbi, axis=1)
labeled_data

,sessionId,timestamp,x,y,z,ppgValue,hr,hrIbi,hrStatus,ibiStatus,notification,engagement,affect,context,label_id,label
12763,1,493272,884.0,1564.0,3767.0,2103379.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,RELAXED
12764,1,493311,900.0,1518.0,3782.0,2104145.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,RELAXED
12765,1,493351,894.0,1553.0,3734.0,2104953.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,RELAXED
12766,1,493391,907.0,1488.0,3729.0,2105398.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,RELAXED
12767,1,493431,861.0,1559.0,3731.0,2105114.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,RELAXED
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8101814,58,6184903,453.0,552.0,4126.0,2126027.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,240,HAPPY
8101815,58,6184943,494.0,607.0,4059.0,2126361.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,240,HAPPY
8101816,58,6184983,695.0,841.0,4124.0,2127234.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,240,HAPPY
8101817,58,6185023,497.0,591.0,4329.0,2128049.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,240,HAPPY


In [5]:
labeled_data['hrIbi'].fillna(method='ffill', inplace=True)
labeled_data['hr'].fillna(method='ffill', inplace=True)
labeled_data['hrIbi'].bfill()
labeled_data['hr'].bfill()
#labeled_data.groupby('hr').head()
nan_in_hr=labeled_data['hr'].isna().any()
nan_count_in_hr = labeled_data['hr'].isna().sum()
if nan_in_hr:
    print("There are NaN values in the 'hr' column after filling.",nan_count_in_hr)

if nan_count_in_hr > 0:
    fill_value = 649
    labeled_data.loc[labeled_data['hrIbi'].isna(), 'hrIbi'] = fill_value
    labeled_data.loc[labeled_data['hr'].isna(), 'hr'] = 95

print("Number of NaN values in the 'hr' column after filling:", labeled_data['hrIbi'].isna().sum())

labeled_data

There are NaN values in the 'hr' column after filling. 19
Number of NaN values in the 'hr' column after filling: 0


,sessionId,timestamp,x,y,z,ppgValue,hr,hrIbi,hrStatus,ibiStatus,notification,engagement,affect,context,label_id,label
12763,1,493272,884.0,1564.0,3767.0,2103379.0,95.0,649.0,NaN,NaN,NaN,NaN,NaN,NaN,1,RELAXED
12764,1,493311,900.0,1518.0,3782.0,2104145.0,95.0,649.0,NaN,NaN,NaN,NaN,NaN,NaN,1,RELAXED
12765,1,493351,894.0,1553.0,3734.0,2104953.0,95.0,649.0,NaN,NaN,NaN,NaN,NaN,NaN,1,RELAXED
12766,1,493391,907.0,1488.0,3729.0,2105398.0,95.0,649.0,NaN,NaN,NaN,NaN,NaN,NaN,1,RELAXED
12767,1,493431,861.0,1559.0,3731.0,2105114.0,95.0,649.0,NaN,NaN,NaN,NaN,NaN,NaN,1,RELAXED
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8101814,58,6184903,453.0,552.0,4126.0,2126027.0,81.0,917.0,NaN,NaN,NaN,NaN,NaN,NaN,240,HAPPY
8101815,58,6184943,494.0,607.0,4059.0,2126361.0,81.0,917.0,NaN,NaN,NaN,NaN,NaN,NaN,240,HAPPY
8101816,58,6184983,695.0,841.0,4124.0,2127234.0,81.0,917.0,NaN,NaN,NaN,NaN,NaN,NaN,240,HAPPY
8101817,58,6185023,497.0,591.0,4329.0,2128049.0,81.0,917.0,NaN,NaN,NaN,NaN,NaN,NaN,240,HAPPY


In [6]:

def preprocess_data(data):
  
    data = data.drop(columns=[ 'notification', 'engagement','context','affect','hrStatus','ibiStatus',])

    scaler = StandardScaler()
    numerical_cols = ['ppgValue','x', 'y', 'z','hr','hrIbi']
    data[numerical_cols] = scaler.fit_transform(data[numerical_cols].ffill())

    label_encoder = LabelEncoder()
    data['label'] = label_encoder.fit_transform(data['label'])

    return data

In [7]:
processed_data = preprocess_data(labeled_data)
processed_data

,sessionId,timestamp,x,y,z,ppgValue,hr,hrIbi,label_id,label
12763,1,493272,0.769742,0.827824,0.279626,-0.198057,1.863799,-0.108217,1,2
12764,1,493311,0.783048,0.800829,0.290323,-0.194905,1.863799,-0.108217,1,2
12765,1,493351,0.778058,0.821369,0.256091,-0.191581,1.863799,-0.108217,1,2
12766,1,493391,0.788869,0.783223,0.252526,-0.189750,1.863799,-0.108217,1,2
12767,1,493431,0.750615,0.824890,0.253952,-0.190918,1.863799,-0.108217,1,2
...,...,...,...,...,...,...,...,...,...,...
8101814,58,6184903,0.411319,0.233920,0.535653,-0.104877,-0.056656,0.466064,240,1
8101815,58,6184943,0.445415,0.266198,0.487871,-0.103503,-0.056656,0.466064,240,1
8101816,58,6184983,0.612568,0.403523,0.534227,-0.099911,-0.056656,0.466064,240,1
8101817,58,6185023,0.447910,0.256808,0.680426,-0.096558,-0.056656,0.466064,240,1


In [8]:
grouped = processed_data.groupby('label_id')
grouped_label =  processed_data.groupby('label')

min = len(grouped.get_group(1))
for i in range(len(grouped)-1):
    if len(grouped.get_group(i+2))<min:
        min = len(grouped.get_group(i+2))

X = []
Y = []
Xa = []
Xb = []
for i in range(len(grouped)):
    X.append(grouped.get_group(i+1)[["x", "y", "z", 'hr', 'hrIbi']][:min])
    Y.append(grouped.get_group(i+1).iloc[0][["label"]])

X = np.array(X)
Y = np.array(Y)
print(X.shape)
print(Y.shape)

(240, 259, 5)
(240, 1)


In [9]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1, random_state=42)

In [57]:

""" model = Sequential([
    Conv1D(32, kernel_size=3, activation='relu', input_shape=(X.shape[1], X.shape[2])),
    MaxPooling1D(pool_size=2),
    Conv1D(64, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid') 
]) """

model = Sequential([
    Conv1D(64, kernel_size=3, activation='relu', input_shape=(X.shape[1], X.shape[2])),
    MaxPooling1D(pool_size=2),
    Conv1D(128, kernel_size=3, activation='relu'),
    Dropout(0.2),  
    MaxPooling1D(pool_size=2),
    Conv1D(256, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Dropout(0.2),  
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.2),  
    Dense(64, activation='relu'),
    Dense(4, activation='softmax') 
])


/home/harsha/bbdc24-mlbananas/venv/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


In [10]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, Y_train, epochs=7, batch_size=64, validation_data=(X_test, Y_test))

loss, accuracy = model.evaluate(X_test, Y_test)
print("Test Accuracy:", accuracy)

NameError: name 'model' is not defined

In [59]:
Y_pred = model.predict(X_test)

Y_pred = np.argmax(Y_pred, axis=1)

from sklearn.metrics import accuracy_score, classification_report

accuracy = accuracy_score(Y_test, Y_pred)
print("Accuracy:", accuracy)

# You can also print a classification report for more detailed evaluation metrics
print(classification_report(Y_test, Y_pred))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
Accuracy: 0.5
              precision    recall  f1-score   support

         1.0       0.70      0.47      0.56        15
         2.0       0.36      0.62      0.45         8
         3.0       0.00      0.00      0.00         1

    accuracy                           0.50        24
   macro avg       0.35      0.36      0.34        24
weighted avg       0.56      0.50      0.50        24



/home/harsha/bbdc24-mlbananas/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/harsha/bbdc24-mlbananas/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/harsha/bbdc24-mlbananas/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitaliz

In [33]:
Y_pred

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0])

In [15]:
from sklearn.model_selection import KFold

k = 10
kf = KFold(n_splits=k, shuffle=True)
accuracies = []
losses = []

# Iterate over each fold
for train_index, test_index in kf.split(X):
    X_train, X_val = X[train_index], X[test_index]
    Y_train, Y_val = Y[train_index], Y[test_index]

    model = Sequential([
        Conv1D(64, kernel_size=3, activation='relu', input_shape=(X.shape[1], X.shape[2])),
        MaxPooling1D(pool_size=2),
        Conv1D(128, kernel_size=3, activation='relu'),
        Dropout(0.2),  
        MaxPooling1D(pool_size=2),
        Conv1D(256, kernel_size=3, activation='relu'),
        MaxPooling1D(pool_size=2),
        Dropout(0.2),  
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.2),  
        Dense(64, activation='relu'),
        Dense(4, activation='softmax') 
    ])

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    history = model.fit(X_train, Y_train, epochs=20, batch_size=64, validation_data=(X_val, Y_val), verbose=0)
    

    loss, accuracy = model.evaluate(X_val, Y_val)
    losses.append(loss)
    accuracies.append(accuracy)

avg_loss = np.mean(losses)
avg_accuracy = np.mean(accuracies)

print(f'Average loss: {avg_loss}')
print(f'Average accuracy: {avg_accuracy}')

/home/harsha/bbdc24-mlbananas/venv/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.4583 - loss: 1.4832
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.6250 - loss: 1.0844
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.5833 - loss: 1.6512
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.4583 - loss: 1.5154
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.6250 - loss: 1.3504
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.6667 - loss: 1.1042
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.5833 - loss: 1.7605
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.5417 - loss: 1.9988
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.5000 - loss: 1.9551
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.4583 - loss: 2.2555
Average loss: 1.61585875749588
Average accuracy: 0.5500000029802322


In [19]:
model.save('ourbest_affect_model.keras')

In [21]:
keras.saving.save_model(model, 'ourbest_affect_model.keras', overwrite=True)

In [16]:
Y_pred = model.predict(X_test)

Y_pred = np.argmax(Y_pred, axis=1)

from sklearn.metrics import accuracy_score, classification_report

accuracy = accuracy_score(Y_test, Y_pred)
print("Accuracy:", accuracy)

# You can also print a classification report for more detailed evaluation metrics
print(classification_report(Y_test, Y_pred))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
Accuracy: 0.9166666666666666
              precision    recall  f1-score   support

         1.0       1.00      0.87      0.93        15
         2.0       0.80      1.00      0.89         8
         3.0       1.00      1.00      1.00         1

    accuracy                           0.92        24
   macro avg       0.93      0.96      0.94        24
weighted avg       0.93      0.92      0.92        24



In [22]:
loaded_model = keras.saving.load_model("ourbest_affect_model.keras")

/home/harsha/bbdc24-mlbananas/venv/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:396: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 14 variables whereas the saved optimizer has 26 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


In [24]:
Y_pred = loaded_model.predict(X_test)

Y_pred = np.argmax(Y_pred, axis=1)

from sklearn.metrics import accuracy_score, classification_report

accuracy = accuracy_score(Y_test, Y_pred)
print("Accuracy:", accuracy)

# You can also print a classification report for more detailed evaluation metrics
print(classification_report(Y_test, Y_pred))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Accuracy: 0.9166666666666666
              precision    recall  f1-score   support

         1.0       1.00      0.87      0.93        15
         2.0       0.80      1.00      0.89         8
         3.0       1.00      1.00      1.00         1

    accuracy                           0.92        24
   macro avg       0.93      0.96      0.94        24
weighted avg       0.93      0.92      0.92        24

